In [1]:
import moxing as mox
mox.file.shift('os', 'mox')
import matplotlib.pyplot as plt

data_path = 's3://obs-wzhtest/cifar/image/vgg_64-512_4.jpg'
def vis_data(stats):

    plt.title('accuracy')
    plt.plot(stats, '-o', label='validation_set')
    plt.axhline(y=0.7, ls=":", c="yellow")  # 添加水平直线
    plt.axhline(y=0.8, ls=":", c="red")
    plt.axis([0, 100, 0, 1])
    plt.legend(loc='lower right')
    plt.xlabel('epoch * 2.6')
    plt.savefig(data_path)
    plt.show()

INFO:root:Using MoXing-v1.17.3-
INFO:root:Using OBS-Python-SDK-3.20.7
/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.


In [2]:
import os
import tensorflow as tf
from data_utils import get_CIFAR10_data
import cifar_cnn, cnn_structure
import numpy as np
import time

%matplotlib inline

batch_size = 64
learning_rate_base, learning_rate_decay = 0.007, 0.995
regularization_rate = 1e-4
iteration = 200001
moving_average_decay = 0.99

model_path = 'model/cifar'
model_name = 'model.ckpt'

def train(x_train, y_train, x_val, y_val, x_test, y_test):

    x = tf.placeholder(tf.float32, [None, cifar_cnn.image_size, cifar_cnn.image_size, cifar_cnn.channels], name='x-input')
    y = tf.placeholder(tf.float32, [None, cifar_cnn.output_size], name='y-input')

    regularizer = tf.contrib.layers.l2_regularizer(regularization_rate)

    # logit = cifar_cnn.forward_propagation(x, True, regularizer) # Lenet Structure
    logit = cnn_structure.vgg_forward_propagation(x, True, regularizer) # vgg structure
    global_step = tf.Variable(0, trainable=False)

    variable_average = tf.train.ExponentialMovingAverage(moving_average_decay, global_step)
    variable_average_op = variable_average.apply(tf.trainable_variables())

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=tf.argmax(y, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)

    loss = cross_entropy_mean + tf.\
        add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(learning_rate_base, global_step,
                                    x_train.shape[0] / batch_size, learning_rate_decay)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variable_average_op]):
        train_op = tf.no_op(name='train')

    correct_prediction = tf.equal(tf.argmax(logit, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        start_time = time.time()
        print('Training start:')
        acc_history = []
        for i in range(iteration):
            # xs, ys = mnist.train.next_batch(batch_size)
            X_batch, y_batch = None, None
            idx = np.random.choice(x_train.shape[0], batch_size, replace=True)
            X_batch = x_train[idx]
            y_batch = y_train[idx]
            _, loss_value, step = sess.run([train_op, loss, global_step],
                                           feed_dict={x:X_batch, y:y_batch})
            

            if i % 2000 == 0:
                val_dict = {x: x_val, y: y_val}
                val_acc = sess.run(accuracy, feed_dict=val_dict)
                acc_history.append(val_acc)
                total_time = time.time() - start_time
                min = int(total_time / 60)
                sec = int(total_time % 60)
                cost_time = str(min) + ' min, ' + str(sec) + ' sec'
                print('After %d iterations, loss: %g, val-acc: %g, time spent: %s' % (i, loss_value, val_acc, cost_time))

            if i % 10000 == 0:
                saver.save(sess, 'model/cifar/model.ckpt',
                           global_step=global_step)
                test_dict = {x: x_test, y: y_test}
                test_acc = sess.run(accuracy, feed_dict=test_dict)
                print('After %d iterations, test_acc: %g' %(i, test_acc))
        
        vis_data(acc_history)
        



In [ ]:
data = get_CIFAR10_data()
x_train, y_train = data['X_train'], data['y_train']
x_val, y_val = data['X_val'], data['y_val']
x_test, y_test = data['X_test'], data['y_test']
train(x_train, y_train, x_val, y_val, x_test, y_test)

Training start:
After 0 iterations, loss: 3.41782, val-acc: 0.098, time spent: 0 min, 1 sec
After 0 iterations, test_acc: 0.0955
After 2000 iterations, loss: 1.54129, val-acc: 0.56, time spent: 0 min, 34 sec
After 4000 iterations, loss: 1.47999, val-acc: 0.677, time spent: 1 min, 4 sec
After 6000 iterations, loss: 1.07608, val-acc: 0.69, time spent: 1 min, 33 sec
After 8000 iterations, loss: 1.12359, val-acc: 0.714, time spent: 2 min, 3 sec
After 10000 iterations, loss: 0.607647, val-acc: 0.787, time spent: 2 min, 32 sec
After 10000 iterations, test_acc: 0.7474
After 12000 iterations, loss: 0.812569, val-acc: 0.762, time spent: 3 min, 2 sec
After 14000 iterations, loss: 0.656642, val-acc: 0.762, time spent: 3 min, 31 sec
After 16000 iterations, loss: 0.831405, val-acc: 0.758, time spent: 4 min, 0 sec
After 18000 iterations, loss: 0.849863, val-acc: 0.774, time spent: 4 min, 30 sec
After 20000 iterations, loss: 0.434518, val-acc: 0.805, time spent: 4 min, 59 sec
After 20000 iterations, 

In [ ]:
data = get_CIFAR10_data()
x_train, y_train = data['X_train'], data['y_train']
x_val, y_val = data['X_val'], data['y_val']
x_test, y_test = data['X_test'], data['y_test']
train(x_train, y_train, x_val, y_val, x_test, y_test)

In [ ]:
data = get_CIFAR10_data()
x_train, y_train = data['X_train'], data['y_train']
x_val, y_val = data['X_val'], data['y_val']
x_test, y_test = data['X_test'], data['y_test']
train(x_train, y_train, x_val, y_val, x_test, y_test)

In [ ]:
data = get_CIFAR10_data()
x_train, y_train = data['X_train'], data['y_train']
x_val, y_val = data['X_val'], data['y_val']
x_test, y_test = data['X_test'], data['y_test']
train(x_train, y_train, x_val, y_val, x_test, y_test)

In [ ]:
data = get_CIFAR10_data()
x_train, y_train = data['X_train'], data['y_train']
x_val, y_val = data['X_val'], data['y_val']
x_test, y_test = data['X_test'], data['y_test']
print('x_test.shape: ', x_test.shape)
train(x_train, y_train, x_val, y_val, x_test, y_test)

In [ ]:
data = get_CIFAR10_data()
x_train, y_train = data['X_train'], data['y_train']
x_val, y_val = data['X_val'], data['y_val']
x_test, y_test = data['X_test'], data['y_test']
print('x_test.shape: ', x_test.shape)
train(x_train, y_train, x_val, y_val, x_test, y_test)